This is a Google Colab notebook which installs appropriate PyTorch v1 according to the system architecture and the GPU available.

If you come across any issues, then feel free to contact me on Slack. My username is ***avinashss***


In [ ]:
# google colab does not come with torch installed. And also, in lesson we are 
# using torch v1.0 
# so following snippet of code installs the relevant version according to the 
# GPU architecture
!pip install -q torch==1.1.0 torchvision
import torch
print(torch.__version__)

In [ ]:
# we will verify that GPU is enabled for this notebook
# following should print: CUDA is available!  Training on GPU ...
# 
# if it prints otherwise, then you need to enable GPU: 
# from Menu > Runtime > Change Runtime Type > Hardware Accelerator > GPU

import torch
import numpy as np

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

In [ ]:
!nvidia-smi

In [ ]:
import torch, argparse
from torchvision import datasets
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

import numpy as np
from collections import OrderedDict

import torch, os
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
def loss_fn(recon_x, x, mu, logvar):
    BCE = F.mse_loss(recon_x, x)
    KLD = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

In [ ]:
class VAE(nn.Module):
    def __init__(self, in_dims=784, hid_dims=100, negative_slope=0.1):
        super(VAE, self).__init__()
        # Encoder
        self.encoder = nn.Sequential(OrderedDict([
            ('layer1', nn.Linear(in_dims, 512)),
            ('relu1', nn.LeakyReLU(negative_slope=negative_slope, inplace=True)),
            ('layer2', nn.Linear(512, 256)),
            ('relu2', nn.LeakyReLU(negative_slope=negative_slope, inplace=True)),
            ('layer3', nn.Linear(256, 128)),
            ('relu3', nn.LeakyReLU(negative_slope=negative_slope, inplace=True)),
        ]))
        self.fc_mu = nn.Linear(128, hid_dims)
        self.fc_var = nn.Linear(128, hid_dims)
        # Decoder
        self.decoder = nn.Sequential(OrderedDict([
            ('layer1', nn.Linear(hid_dims, 128)),
            ('relu1', nn.LeakyReLU(negative_slope=negative_slope, inplace=True)),
            ('layer2', nn.Linear(128, 256)),
            ('relu2', nn.LeakyReLU(negative_slope=negative_slope, inplace=True)),
            ('layer3', nn.Linear(256, 512)),
            ('relu3', nn.LeakyReLU(negative_slope=negative_slope, inplace=True)),
            ('layer4', nn.Linear(512, in_dims)),
            ('sigmoid', nn.Sigmoid()),
        ]))
        self._init_weights()

    def forward(self, x):
        if self.training:
            h = self.encoder(x)
            mu, logvar = self.fc_mu(h), self.fc_var(h)
            z = self._reparameterize(mu, logvar)
            y = self.decoder(z)
            return y, mu, logvar
        else:
            z = self.represent(x)
            y = self.decoder(z)
            return y

    def represent(self, x):
        h = self.encoder(x)
        mu, logvar = self.fc_mu(h), self.fc_var(h)
        z = self._reparameterize(mu, logvar)
        return z

    def _reparameterize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        esp = torch.randn(*mu.size()).type_as(mu)
        z = mu + std * esp
        return z

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

In [ ]:
class ImproveChecker():
	def __init__(self, mode='min', best_val=None):
		assert mode in ['min', 'max']
		self.mode = mode
		if best_val is not None:
			self.best_val = best_val
		else:
			if self.mode=='min':
				self.best_val = np.inf
			elif self.mode=='max':
				self.best_val = 0.0

	def check(self, val):
		if self.mode=='min':
			if val < self.best_val:
				print("[%s] Improved from %.4f to %.4f" % (self.__class__.__name__, self.best_val, val))
				self.best_val = val
				return True
			else:
				print("[%s] Not improved from %.4f" % (self.__class__.__name__, self.best_val))
				return False
		else:
			if val > self.best_val:
				print("[%s] Improved from %.4f to %.4f" % (self.__class__.__name__, self.best_val, val))
				self.best_val = val
				return True
			else:
				print("[%s] Not improved from %.4f" % (self.__class__.__name__, self.best_val))
				return False

In [ ]:
# Initialize VAE
model = VAE(in_dims=784, hid_dims=100)
model.cuda()

# Configure data loader
dataset = datasets.MNIST(root='.', train=True, download=True,
	transform=transforms.Compose([
		transforms.ToTensor(),
		transforms.Normalize((0.1307,), (0.3081,))
]))
dataloader = torch.utils.data.DataLoader(
	dataset, batch_size=128,
	num_workers=4, shuffle=True, pin_memory=True,
)

# Optimizers
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# ImproveChecker
improvechecker = ImproveChecker(mode='min')

In [ ]:
model.train()
for epoch in range(1, 301):
	for i, (imgs, _) in enumerate(dataloader):
		# Prepare input
		inputs = imgs.view(imgs.shape[0], -1)
		inputs = inputs.cuda()

		# Train
		optimizer.zero_grad()
		outputs, mu, logvar = model(inputs)
		loss = loss_fn(outputs, inputs, mu, logvar)
		loss.backward()
		optimizer.step()

	# ImproveChecker
	print("[EPOCH %.3d] Loss: %.6f" % (epoch, loss.item()))
	if improvechecker.check(loss.item()):
		checkpoint = dict(
			epoch=epoch,
			loss=loss.item(),
			state_dict=model.state_dict(),
			optimizer=optimizer.state_dict(),
		)
		save_file = os.path.join('.', "vae.pth")
		torch.save(checkpoint, save_file)
		print("Best checkpoint is saved at %s" % (save_file))
